# K-MCI
### Intro
In this notebook we have the main algorithm of the article. All necessary functions are implemented in function.py file.<br>
This document have blow sections:<br>
1. Importing Libraries & classes
2. Reading datasets
3. Preprocessing datasets
4. Main Algorithm

### Importing Libraries & classes
We import some libraries from third party.

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import random
import copy
from sklearn import preprocessing

In [2]:
# defnition of Candidate class
class Candidate:    
    def __init__(self,nodes,sampling_interval,fitness=-1):
        self.nodes = nodes
        self.sampling_interval = sampling_interval 
        self.variation = []
        self.random_number
        self.fitness = fitness
        self.centers = []
        self.probability = []
        self.labels = []
        self.each_fitness = []
        
    def random_number(self):
        self.random_number = random.random()
        
    def describe(self):
        print("Candidate with #{} nodes, #{} centers, fitness={}\n clusters:\n{} \n Centers:\n{}\n sampling_interval:\n{}"
              .format(len(self.nodes),len(self.centers),self.fitness,self.nodes[np.random.randint(self.nodes.shape[0], size=5), :],self.centers,self.sampling_interval))

### Reading Datasets and Preprocessing
We show the structure of our data for all of datasets.<br>
We have these datasets from UCI Machine Learning Repository:
1. Breast Cancer Wisconsin
2. Contraceptive Method Choice data
3. Glass
4. Iris
5. Vowel
6. Wine

In [3]:
""" In this cell we have some code which will do preprocessing for us on all datasets """
""" the resourses of these datasets are mentioned in README.md file. """

path = 'datasets/' 
# because we are using local files, you need to download datasets and change the "path" variable
# local folder of your downloaded datasets


def bcw(): 
    # importing dataset
    dataset = pd.read_csv(path+'bcw.csv',
                          names = ['Sample code number','Clump Thickness','Uniformity of Cell Size',
                                   'Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial',
                                   'Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class'])
    dataset = dataset.drop(dataset.columns[0], axis=1)
    dataset= dataset.replace(to_replace='?',value=np.nan)
    dataset = dataset.dropna(axis =0) # resolving missing values
    dataset = dataset.astype('int64') # casting string valued column to int64
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def cmc():
    # importing dataset
    dataset = pd.read_csv(path+'cmc.csv', names=["Wife's age","Wife's education","Husband's education",
                                                 "Number of children ever born","Wife's religion",
                                                 "Wife's now working?","Husband's occupation",
                                                 "Standard-of-living index","Media exposure","Contraceptive method used"])
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def glass():
    # importing dataset
    dataset = pd.read_csv(path+'glass.csv', names= ['Id','refractive index','Sodium','Magnesium','Aluminum','Silicon','Potassium'
                                                    ,'Calcium','Barium','Iron','glass'])
    x = dataset.iloc[:,1:10].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def iris():
    # importing dataset
    dataset = pd.read_csv(path+'iris.csv', names= ['sepal length','sepal width','petal length','petal width','class'])
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    
    # encoding categorial data types to labelEncoder
    from sklearn.preprocessing import LabelEncoder
    labelencoder_y = LabelEncoder()
    labelencoder_y = labelencoder_y.fit(y)
    y = labelencoder_y.transform(y)  # 0 for 'Iris-setosa', 1 for 'Iris-versicolor', 2 for 'Iris-virginica'
    return (x,y,dataset)


def vowel():
    # importing dataset
    dataset = pd.read_csv(path+'vowel.csv', names= ['vowel','type 1 frq','type 2 frq','type 3 frq'])
    x = dataset.iloc[:,1:].values # features
    y = dataset.iloc[:,0].values # target values
    return (x,y,dataset)

def wine():
    # importing dataset
    dataset = pd.read_csv(path+'wine.csv', names= ['class','Alcohol','Malic acid','Ash','Alcalinity of ash','Magnesium','Total phenols','Flavanoids','Nonflavanoid phenols','Proanthocyanins','Color intensity','Hue','OD280/OD315','Proline'])
    x = dataset.iloc[:,1:].values # features
    y = dataset.iloc[:,0].values # target values
    return (x,y,dataset)

In [4]:
# Getting data from our preprocessing class Datasets
x_bcw, y_bcw, df_bcw = bcw()
x_cmc, y_cmc, df_cmc = cmc()
x_glass, y_glass, df_glass = glass()
x_iris, y_iris, df_iris = iris()
x_vowel, y_vowel, df_vowel = vowel()
x_wine, y_wine, df_wine = wine()

In [5]:
# Showing dataset of Breast Cancer Wisconsin
df_bcw.head(3)

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2


In [6]:
# Showing dataset of Contraceptive Method Choice data
df_cmc.head(3)

,Wife's age,Wife's education,Husband's education,Number of children ever born,Wife's religion,Wife's now working?,Husband's occupation,Standard-of-living index,Media exposure,Contraceptive method used
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1


In [7]:
# Showing dataset of Glass
df_glass.head(3)

,Id,refractive index,Sodium,Magnesium,Aluminum,Silicon,Potassium,Calcium,Barium,Iron,glass
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1


In [8]:
# Showing dataset of Iris
df_iris.head(3)

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


In [9]:
# Showing dataset of Vowel
df_vowel.head(3)

,vowel,type 1 frq,type 2 frq,type 3 frq
0,1,700,1500,2600
1,1,550,1550,2400
2,1,700,1500,2600


In [10]:
# Showing dataset of Wine
df_wine.head(3)

,class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185


### Standardization
In this cell, we standard our data using this equation: <img src='http://3.bp.blogspot.com/_xqXlcaQiGRk/RpO4yR0oKtI/AAAAAAAAABM/7rUWCMwizus/s200/fig2.png'></img>

In [11]:
x_bcw = preprocessing.scale(x_bcw)
x_wine = preprocessing.scale(x_wine)
x_cmc = preprocessing.scale(x_cmc)
#x_glass = preprocessing.scale(x_glass)
x_iris = preprocessing.scale(x_iris)
x_vowel = preprocessing.scale(x_vowel)

C:\Users\HaMeD\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


# Main Algorithm 

The implementation of main algorithm have below sections:
1. <Strong>Initializing Parameters and Candidates</Strong>
    1. Sampling Interval
    2. Apply Random Centers to Candidates
    3. Visualizing Candidates
2. <Strong>Running Kmean to Exploit</Strong>
    1. K-means Algorithm
    2. Visualizing Candidates
3. <Strong>Mutation Algorithm</Strong>
    1. Fitness over Mutated Candidate
    2. Probablility Equation
    3. Mutation Logic

**Extra details will be explain in other next sections**


## Initializing Parameters and Candidates
In this step, we initialize our algorithm. There are some parameters which are important to convergance speed and quality of solution. So we initialize them here based on the paper.<br>
Each time you want to run K-MCI algorithm, you should first do this initialization.<br>

In [12]:
# notice: this values related to IRIS dataset
candidates_array = [] # the array of all candidates
number_of_candidates = 5
sampling_interval_reduction_factor = 0.95
convergence_parameter = None # what is this in paper???
mutation_random = 0.7
iterations_count = 3500
variations_count = 15
number_of_clusters = 3 # in this article number of clusters are predefined

## Sampling Interval
this function use for making sampling_interval


In [13]:
def sampling_intervals(input_array):
    Maxes = np.amax(input_array,axis=0)
    Mins  = np.amin(input_array,axis=0)
    return (Maxes,Mins)

In [14]:
# initializing candidates
for i in range(0 ,number_of_candidates):
    candidates_array.append(Candidate(x_glass,sampling_interval = sampling_intervals(x_glass)))

## Visualizing Candidates
some parameters are uninitialized. They will have value after running Kmean and Cohort and mutations.

In [15]:
candidates_array[random.randint(0,number_of_candidates-1)].describe()

Candidate with #214 nodes, #0 centers, fitness=-1
 clusters:
[[ 1.52068 13.55     2.09     1.67    72.18     0.53     9.57     0.27
   0.17   ]
 [ 1.52152 13.05     3.65     0.87    72.22     0.19     9.85     0.
   0.17   ]
 [ 1.51708 13.72     3.68     1.81    72.06     0.64     7.88     0.
   0.     ]
 [ 1.51934 13.64     3.54     0.75    72.65     0.16     8.89     0.15
   0.24   ]
 [ 1.51316 13.02     0.       3.04    70.48     6.21     6.96     0.
   0.     ]] 
 Centers:
[]
 sampling_interval:
(array([ 1.53393, 17.38   ,  4.49   ,  3.5    , 75.41   ,  6.21   ,
       16.19   ,  3.15   ,  0.51   ]), array([ 1.51115, 10.73   ,  0.     ,  0.29   , 69.81   ,  0.     ,
        5.43   ,  0.     ,  0.     ]))


## Apply Random Centers to Candidates
In this step, we just select some random nodes as centers with psuedorandom functions in python native library.

In [16]:
for c in range(0,number_of_candidates):
    candidates_array[c].centers =  np.asarray(random.sample(list(candidates_array[c].nodes),number_of_clusters))
    print(candidates_array[c].centers)

[[ 1.52177 13.75     1.01     1.36    72.19     0.33    11.14     0.
   0.     ]
 [ 1.51629 12.71     3.33     1.49    73.28     0.67     8.24     0.
   0.     ]
 [ 1.51839 12.85     3.67     1.24    72.57     0.62     8.68     0.
   0.35   ]]
[[ 1.53125 10.73     0.       2.1     69.81     0.58    13.3      3.15
   0.28   ]
 [ 1.51316 13.02     0.       3.04    70.48     6.21     6.96     0.
   0.     ]
 [ 1.52081 13.78     2.28     1.43    71.99     0.49     9.85     0.
   0.17   ]]
[[ 1.51631 13.34     3.57     1.57    72.87     0.61     7.89     0.
   0.     ]
 [ 1.51667 12.94     3.61     1.26    72.75     0.56     8.6      0.
   0.     ]
 [ 1.51719 14.75     0.       2.      73.02     0.       8.53     1.59
   0.08   ]]
[[ 1.51755 13.       3.6      1.36    72.99     0.57     8.4      0.
   0.11   ]
 [ 1.52475 11.45     0.       1.88    72.19     0.81    13.24     0.
   0.34   ]
 [ 1.53125 10.73     0.       2.1     69.81     0.58    13.3      3.15
   0.28   ]]
[[ 1.51674 12.79  

## K-means Algorithm 
this function fitting k-means model on dataset

In [17]:
def doKmeans(x,clusters_count,init_centers):
    kmeans = KMeans(n_clusters = clusters_count, init = init_centers,n_init = 50)
    kmeans = kmeans.fit(x)
    labels = kmeans.labels_
    return (labels,kmeans.cluster_centers_,kmeans.inertia_)


## Running Kmean to Exploit
Here by running Kmean on all of our candidates, they centers initialized and nodes assigns to clusters based on Kmean++ algorithm.<br>
The objective function of Kmean is also the object function of K-MCI. <br>
So the fitness of any candidates is calculated using Kmean inertia or mean square error of nodes to their centers distances.

In [18]:

for c in range(0,number_of_candidates):
    y_predict_temp,centers_temp,fitness_temp = doKmeans(candidates_array[c].nodes,clusters_count = number_of_clusters,init_centers = candidates_array[c].centers)
    candidates_array[c].centers = centers_temp
    candidates_array[c].fitness = fitness_temp
    candidates_array[c].labels   = y_predict_temp
    
candidates_array[2].labels

C:\Users\HaMeD\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=50
  return_n_iter=True)


array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2,
       1, 1, 1, 1, 0, 2, 2, 2, 2, 0, 0, 0, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

## Visualizing Candidates
In this step, by Kmeans, we assigned centers to each candidates and calculated the fitness of each candidate using inertia

In [19]:
candidates_array[random.randint(0,number_of_candidates-1)].describe()

Candidate with #214 nodes, #3 centers, fitness=735.3256911794765
 clusters:
[[ 1.51784 13.08     3.49     1.28    72.86     0.6      8.49     0.
   0.     ]
 [ 1.52171 11.56     1.88     1.56    72.86     0.47    11.41     0.
   0.     ]
 [ 1.51685 14.92     0.       1.99    73.06     0.       8.4      1.59
   0.     ]
 [ 1.51768 12.56     3.52     1.43    73.15     0.57     8.54     0.
   0.     ]
 [ 1.51818 13.72     0.       0.56    74.45     0.      10.99     0.
   0.     ]] 
 Centers:
[[1.51715800e+00 1.31121667e+01 3.46491667e+00 1.38158333e+00
  7.28745833e+01 5.79250000e-01 8.33916667e+00 3.51666667e-02
  6.10833333e-02]
 [1.52035413e+00 1.38308696e+01 3.22130435e+00 1.25195652e+00
  7.17384783e+01 5.37608696e-01 9.17304348e+00 1.07608696e-01
  5.36956522e-02]
 [1.51947813e+00 1.37416667e+01 2.19166667e-01 1.78812500e+00
  7.29662500e+01 2.52708333e-01 1.02943750e+01 5.89375000e-01
  5.00000000e-02]]
 sampling_interval:
(array([ 1.53393, 17.38   ,  4.49   ,  3.5    , 75.41   , 

In [22]:
sum_of_squares(candidates_array[0].nodes,candidates_array[0].centers,candidates_array[0].labels)

726.3151680575362

## Fitness over Mutated Candidate
Just simple sum of square of distances

In [23]:
def sum_of_squares(data, centroids, labels):
    sqe = 0
    for l in np.unique(labels):
        data_l = data[labels == l]
        resid = data_l - centroids[l]
        sqe += (resid**2).sum()
    return sqe

### Mean Square Distance to a Specific Center
This function is exactly based on "sum_of_squares" function. But here all centers not engaged because we want to find the sum of square distances of nodes of a cluster to their cluster center.<br>
The idea is when mutation process done, at least one dimension may get better center but the other goes bad. So we want to have that center which is better to the previous time.

In [24]:
def center_dist(data, center, labels, center_position, center_cluster):
    sqe = 0
    for l in range(0,len(labels)):
        if center_cluster == labels[l] :
            data_l = data[l][center_position]
            resid = data_l - center
            sqe += (resid**2)
    return sqe

In [25]:
def each_fitness(candidates_array_example):
    fitness_of_each_centers = []
    for i in range(0,len(candidates_array_example.centers)):
        Second_layer = []
        for j in range(0,len(candidates_array_example.centers[i])):
            Second_layer.append(center_dist(candidates_array_example.nodes,candidates_array_example.centers[i][j],candidates_array_example.labels,j,i))
        fitness_of_each_centers.append(Second_layer)
    return np.array(fitness_of_each_centers)
print(each_fitness(candidates_array[0]))

[[9.51473731e-04 8.59802667e+01 1.37273667e+01 1.87937312e+01
  4.45367250e+01 1.05977479e+01 2.22138181e+02 2.95950812e+01
  5.49000000e-01]
 [2.80532493e-04 2.35398031e+01 3.74892892e+01 1.67442223e+01
  4.04513392e+01 6.82689723e+01 4.34104531e+01 9.23969308e+00
  1.18195077e+00]
 [1.46307722e-04 1.02577889e+01 1.57234306e+01 5.53229722e+00
  1.11642972e+01 1.32568889e+00 1.26072750e+01 3.17238889e+00
  2.86800000e-01]]


# test paper own answer
## result:
**paper has wrong answer and numbers**

In [28]:
def just_test(data,center):
    temp1 = np.subtract(center,data)
    temp1 = np.power(temp1, 2)
    sqe = np.sum(temp1)
    return sqe


center =[[1.52434 , 12.03344 , 0.01215 , 1.12869 , 71.98256 , 0.19252 , 14.34306 , 0.23039 , 0.15156],
         [1.51956 , 13.25068 , 0.45229 , 1.53305 , 73.01401 , 0.38472 , 11.15803 , 0.00433 , 0.06599],
         [1.51362 , 13.15690 , 0.65548 , 3.13123 , 70.50411 , 5.33024 , 6.73773  , 0.67322 , 0.01490],
         [1.52132 , 13.74692 , 3.51952 , 1.01524 , 71.89517 , 0.21094 , 9.44764  , 0.03588 , 0.04680],
         [1.51933 , 13.08412 , 3.52765 , 1.36555 , 72.85826 , 0.57913 , 8.36271  , 0.00837 , 0.06182],
         [1.51567 , 14.65825 , 0.06326 , 2.21016 , 73.25324 , 0.02744 , 8.68548  , 1.02698 , 0.00382]]

label = []
for i in x_glass:
    test = []
    for j in center:
        test.append(just_test(i,j))
    label.append(np.argmin(test))
    
print(label)    
print(sum_of_squares(x_glass,center,label))
print(candidates_array[0].fitness)

[3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 3, 4, 4, 4, 3, 4, 3, 4, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 0, 0, 0, 1, 1, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 1, 1, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 3, 3, 3, 3, 5, 5, 1, 1, 5, 4, 4, 4, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
343.7814274034
726.3151680575362


## Mutation algorithm 
this function do mutation algorithm on candidates interval's centers

In [29]:
def mutation(candidate_array,mutation_random):
    New_candidate = copy.deepcopy(candidate_array)
    Mutant_candidate = copy.deepcopy(candidate_array)
    Trial_candidate = copy.deepcopy(candidate_array)
    for x in range(len(candidate_array)):
        a = np.full(len(candidate_array), 1/(len(candidate_array)-1))
        a[x] = 0
        temp = np.random.choice(len(candidate_array), 3, replace = False, p=a)
        for i in range(len(candidate_array[0].centers)):
            for j in range(len(candidate_array[0].centers[i])):
                Mutant_candidate[x].centers[i][j] = candidate_array[temp[0]].centers[i][j]+ random.random()*(candidate_array[temp[1]].centers[i][j] - candidate_array[temp[2]].centers[i][j])
                if random.random() < mutation_random:
                    Trial_candidate[x].centers[i][j] = Mutant_candidate[x].centers[i][j] #Trial & Mutant must be copy of same candidate
                    
                    
        Trial_candidate[x].fitness = sum_of_squares(Trial_candidate[x].nodes,Trial_candidate[x].centers,Trial_candidate[x].labels)
        
        if Trial_candidate[x].fitness < sum_of_squares(candidate_array[x].nodes,candidates_array[2].centers,candidates_array[2].labels):
            New_candidate[x] = Trial_candidate[x]
            print("sakjfnr")
        else:
            New_candidate[x] = candidate_array[x]
        #print(Trial_candidate[x].centers)
        #print(candidate_array[x].centers)
        #print(New_candidate[x].centers)
        #print(Mutant_candidate[x].centers)
        print("###########################################################################################################")
        Trial_candidate[x].each_fitness = each_fitness(Trial_candidate[x])
        print(Trial_candidate[x].each_fitness)
        candidate_array[x].each_fitness = each_fitness(candidate_array[x])
        print(candidate_array[x].each_fitness)
        print("###########################################################################################################")
        
    return New_candidate
#print(candidates_array[1].fitness)
#for i in range(0,3500):
New_candidates_array = mutation(candidates_array,mutation_random)
    #if New_candidates_array[1].fitness != candidates_array[1].fitness:
     #   print(New_candidates_array[1].fitness)#never happen

###########################################################################################################
[[1.04523880e-03 1.14179128e+02 4.69071093e+02 1.87937312e+01
  4.99497413e+01 1.28563750e+01 3.41099572e+02 4.04006737e+01
  5.49000000e-01]
 [2.08027721e-03 3.36496794e+01 9.13271758e+02 1.67442223e+01
  4.78749621e+02 3.26795493e+02 3.11543436e+02 9.51452395e+00
  1.18195077e+00]
 [5.87872899e-04 1.02577889e+01 1.00274086e+02 1.55633946e+01
  5.53588230e+01 2.48594023e+00 1.26072750e+01 3.17238889e+00
  2.86956721e-01]]
[[9.51473731e-04 8.59802667e+01 1.37273667e+01 1.87937312e+01
  4.45367250e+01 1.05977479e+01 2.22138181e+02 2.95950812e+01
  5.49000000e-01]
 [2.80532493e-04 2.35398031e+01 3.74892892e+01 1.67442223e+01
  4.04513392e+01 6.82689723e+01 4.34104531e+01 9.23969308e+00
  1.18195077e+00]
 [1.46307722e-04 1.02577889e+01 1.57234306e+01 5.53229722e+00
  1.11642972e+01 1.32568889e+00 1.26072750e+01 3.17238889e+00
  2.86800000e-01]]
######################################

# Probability Equation

In [30]:
def probability(candidates):
    for c in range(0,len(candidates)):
        temp_array = []
        for clusters in range(0,len(candidates[c].centers)):
            temp_array.append((1/np.sum(candidates[c].each_fitness[clusters]))/(np.sum(1/(np.sum(x.each_fitness[clusters])) for x in candidates)))
        print(temp_array)    
        candidates[c].probability = temp_array
    return

### Roulette Wheel Selection
This is a logic for selecting targets with higher probability.<br>
In this article, each candidate want to follow the other candidate with higher fitness. So in the previous section <strong><em>Probability Equation<strong><em>, we calculated all probabilities of candidates based on their fitnesses.

In [31]:
def roulette_wheel_selection(inertia_array):
    maximum = np.sum(inertia_array)
    pick = random.uniform(0, maximum)
    current = 0
    for fitness in inertia_array:
        current += fitness
        if current > pick:
            return fitness

# Shrink sampling interval
**here we shrink sampeling interval using reduction factor ro creat neighborhood of each center**

In [32]:
def shrinked_sampling_interval(candidates_array):
    for i in range(0,len(candidates_array)):
        shrinked_sampling_interval = []
        for j in range(0,len(candidates_array[i].centers)):
            Second_layer = []
            for k in range(0,len(candidates_array[i].centers[j])):
                Third_layer = []
                Temp = abs(candidates_array[i].sampling_interval[0][k] - candidates_array[i].sampling_interval[1][k]) * sampling_interval_reduction_factor
                Third_layer.append(candidates_array[i].centers[j][k] - (Temp/2))
                Third_layer.append(candidates_array[i].centers[j][k] + (Temp/2))
                Second_layer.append(Third_layer)
            shrinked_sampling_interval.append(Second_layer)
        candidates_array[i].sampling_interval = shrinked_sampling_interval
    return
shrinked_sampling_interval(New_candidates_array)
print(New_candidates_array[0].sampling_interval)

[[[1.5086576250000001, 1.530298625], [10.582916666666668, 16.900416666666665], [-1.9135833333333352, 2.351916666666665], [0.26337500000000036, 3.312875], [70.30625, 75.62625], [-2.697041666666667, 3.202458333333333], [5.183375, 15.405375000000001], [-0.9068749999999999, 2.085625], [-0.19225, 0.29225]], [[1.5063012692307693, 1.5279422692307691], [9.99248076923077, 16.30998076923077], [1.2781730769230761, 5.543673076923076], [-0.09021153846153829, 2.9592884615384616], [70.10592307692308, 75.42592307692307], [-2.2762115384615385, 3.6232884615384613], [3.1927692307692297, 13.414769230769231], [-1.4480192307692308, 1.544480769230769], [-0.1816346153846154, 0.3028653846153846]], [[1.5105522777777778, 1.5321932777777776], [10.730694444444445, 17.048194444444444], [1.2158611111111113, 5.481361111111111], [-0.5000277777777777, 2.5494722222222217], [69.15527777777778, 74.47527777777778], [-2.764194444444444, 3.1353055555555556], [4.4215, 14.643500000000001], [-1.4156944444444444, 1.5768055555555

# variations
**Choose t diffrent random center of each cluster's center in shrinked sampling interval neghborhood**

In [33]:
def variation(candidates_array):
    for candidate in candidates_array:
        variations = []
        for i in range(0,variations_count):
            Second_layer = []
            for j in candidate.sampling_interval:
                Third_layer = []
                for k in range(0,len(j)):
                    #print(k)
                    Third_layer.append(random.uniform(j[k][0],j[k][1]))
                Second_layer.append(Third_layer)
            variations.append(Second_layer)
        candidate.variation = variations    
variation(New_candidates_array)
print(New_candidates_array[3].variation[1])

[[1.5175153353916244, 11.729301628824151, 1.6406132230396786, 2.4495249219008146, 75.14132909880509, -1.5779982306186213, 9.054213000792057, 0.9680091113323939, 0.29647966527147795], [1.526857766055257, 11.622566365597454, 1.4994282837582409, 2.122926008536992, 74.57102736085074, -1.5305636394748507, 8.91231421382135, 1.9976763193521925, 0.07331722237031835], [1.5193003550625679, 9.647635745644672, 0.434020219541305, 2.293684645289213, 71.22196557247841, 0.2706955747650315, 14.104891701718913, -1.1285823581602674, -0.06151145601390137]]
